In [ ]:
from importlib import reload
reload(change)
reload(base)

In [1]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [2]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [3]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [4]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [8]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [9]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [10]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [11]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [58]:
ab_grup=base.groupby_date_time(df_change['6AU8_6BU8'])

In [60]:
ab_lst= list(ab_grup)

In [75]:
for i in range(len(ab_lst)):
    sc=ab_lst[i][1]
    sc=base.to_series(sc)
    x=change.get_amplitude(sc)
    print(x.duration.min())

0 days 00:00:01
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:01
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:01
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00


In [76]:
sc=ab_lst[1][1]

In [77]:
sc=base.to_series(sc)

In [84]:
sc=base.clean_data(sc)

In [88]:
signed_sc=change.sign(sc)

In [90]:
signed_scr = signed_sc.reset_index()

In [95]:
signed_scr['duration']=signed_scr.date.diff()

In [101]:
signed_scr

,date,6AU8_6BU8,sign,duration
0,2018-09-06 01:00:03,-0.000096,1,NaT
1,2018-09-06 01:00:40,0.000096,2,00:00:37
2,2018-09-06 01:00:47,-0.000096,3,00:00:07
3,2018-09-06 01:00:56,0.000050,4,00:00:09
4,2018-09-06 01:00:58,-0.000050,5,00:00:02
...,...,...,...,...
532,2018-09-06 01:59:52,0.000013,404,00:00:09
533,2018-09-06 01:59:53,0.000046,404,00:00:01
534,2018-09-06 01:59:54,-0.000097,405,00:00:01
535,2018-09-06 01:59:57,-0.000013,405,00:00:03


In [103]:
amp=signed_scr.groupby('sign').agg({'date':change.last_time,'duration':'sum','6AU8_6BU8':'sum'})

In [109]:
amp

,date,duration,6AU8_6BU8
sign,,,
1,2018-09-06 01:00:03,00:00:00,-0.000096
2,2018-09-06 01:00:40,00:00:37,0.000096
3,2018-09-06 01:00:47,00:00:07,-0.000096
4,2018-09-06 01:00:56,00:00:09,0.000050
5,2018-09-06 01:00:58,00:00:02,-0.000050
...,...,...,...
401,2018-09-06 01:59:40,00:00:01,-0.000059
402,2018-09-06 01:59:41,00:00:01,0.000059
403,2018-09-06 01:59:43,00:00:02,-0.000059


In [19]:
all_pairs_of_amplitude=[]

In [20]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change])) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

#### 1. Amplitude'u hesaplanan tüm pair'lar all_pairs_of_amplitude adlı dizide tutuluyor.  Hepsi tek DataFrame'de birleştirilmek istenirse aşağıdaki kod çalıştırılabilir.

In [ ]:
from functools import reduce

df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

#### 2. Henüz condition uygulanmamış sonuçlar excel'e yazdırabilir.

In [ ]:
df_mid_price.to_excel('out/mid_price.xlsx',)
df_spread.to_excel('out/spread.xlsx')
df_change.to_excel('out/change.xlsx')
df_final.to_excel('out/amplitude.xlsx')

# Koşula Göre Amplitude ve Duration Verisinin Yeniden Düzenlenmesi

In [34]:
#Önce condition modülümüzü import ediyoruz.
import src.condition as condition

In [39]:
# Hesaplayacağımız verileri saklayabilmek için boş bir dizi yarattık
condition_data = []

In [41]:
# Python'a ait lambda ifadelerini kullanabilmek için basit bir fonksiyon yazdık
def uygula(data):
    """parametre olarak gelen 'data' daki indeksi kaldırır.
       percentile(.6) ya göre koşul uygulandı.
       percentile silinip yerine medyan() yazılırsa, koşulu medyana göre hesaplar.
       ya da percentile'ın '.6' olan parametresi değiştirilebilir.
    """
    data = data.reset_index(drop=True)
    signs = condition.Sign(data).percentile(.6) #
    return condition.apply(data,signs)

In [68]:
# bir for döngüsü ile tüm pairlara erişiyoruz
for data in all_pairs_of_amplitude:
    #groupby içerisindeki floor metoduyla veriyi neye göre gruplayacağımızı belirtiyoruz.
    #Burada groupby 2 parametre aldı. ilk parametre ile 'güne göre', 2. parametre ile de 'saate' göre
    #ayırıp, 'uygula' metodunu her gruba ayrı ayrı uygulayacağını belirttik
    """ 'd' -> day
        '2h' -> 2şer saat
        'h'-> birer saat
        'nd' ->n güne böl
        'nh' -> n saate böl gibi        
    """ 
    """.apply metodu ile yapmak istediğimiz işlemi belirtiyoruz
        lambda x : uygula(x) -> her bir grubu sırayla
        x olarak adlandır ve uygula fonksiyonuna parametre olarak gönder
    """
    result = data.groupby([data.date.dt.floor('d'),data.date.dt.floor('2h')])\
        .apply(lambda x : uygula(x)).droplevel([0,1])
    #son olarak hesaplanan verileri condition_data adlı listeye atmasını söylüyoruz
    condition_data.append(result)

duration= {'pozitif': 7.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.8999999999809205e-05}
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 0.00010899999999969268, 'negatif': -9.649999999994385e-05}
duration= {'pozitif': 4.0, 'negatif': nan}
amplitude= {'pozitif': 0.00010899999999991472, 'negatif': -5.900000000014227e-05}
duration= {'pozitif': 12.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.900000000003125e-05}
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.8999999999809205e-05}
duration= {'pozitif': 6.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.0000000000105516e-05}
duration= {'pozitif': 8.0, 'negatif': nan}
amplitude= {'pozitif': 6.150000000004763e-05, 'negatif': -6.14999999999366e-05}
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999976694e-05, 'negatif': -6.14999999999366e-05}
dura

In [71]:
# percentile(0.6)'a göre hesaplanmış olan 1. pairi görüntüleyelim

In [72]:
condition_data[0]

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:13,-0.000218
1,2018-09-06 00:00:39,00:00:22,0.000050
2,2018-09-06 00:00:53,00:00:14,-0.000109
3,2018-09-06 00:01:01,00:00:08,0.000096
4,2018-09-06 00:01:22,00:00:21,-0.000159
...,...,...,...
347,2018-09-07 23:56:50,00:00:16,-0.000062
348,2018-09-07 23:57:25,00:00:35,0.000062
349,2018-09-07 23:58:50,00:01:25,-0.000200
350,2018-09-07 23:58:54,00:00:04,0.000111
